In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from utils import (
    DEGREES_PER_PIXEL,
    TIMESTAMP_IDENT,
    X_PIXELS,
    Y_PIXELS,
    Events,
    extract_gaze_data_between_timestamps_proper,
    get_participant_dominant_eye,
)
import json


from velocityThreshold import detect_fix_ivt, find_sacc_from_fix

# from scipy import ttest_ind

EYE_TRACKER_FOLDER = "eye_tracker_data/"
GAZE_DATA = []
GAZE_DATA_BOOK = []
GAZE_DATA_PAGE = []

# participants that were told that they can change the settings beforehand
participant_ids = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
subquery = Events.select().where(Events.participant_id.in_(participant_ids))

for participant in participant_ids:
    books = subquery.where(
        (Events.event == "OPEN_BOOK")
        & (Events.new_value != "Chasing Sunsets - ")
        & (Events.participant_id == participant)
    )

    saccades = []
    for book in books:
        START_TIME_BOOK = book.time
        book_end = (
            subquery.where(Events.event == "CLOSE_BOOK")
            .where(Events.time > START_TIME_BOOK)
            .get()
        )
        END_TIME_BOOK = book_end.time

        # Get the events that have a timestamp less than the first CLOSE_BOOK event
        events_book = subquery.where(Events.time <= END_TIME_BOOK)

        # Read from participants.json whether or not the participant has low resolution data
        # These we should not divide the timestamp with 1000

        LOW_RES = json.load(open("participants.json", "r"))[f"{book.participant_id}"][
            "low_resolution"
        ]
        if not LOW_RES:
            START_TIME_BOOK /= 1000
        formatted_time = datetime.fromtimestamp(START_TIME_BOOK).strftime(
            "%Y-%m-%d_%H-%M-%S"
        )
        GAZE_FILE = f"{EYE_TRACKER_FOLDER}[{book.participant_id}]-{formatted_time}.json"
        f = open(GAZE_FILE, "r")
        GAZE_DATA_BOOK = json.load(f)
        f.close()

        timestamps = []
        x = []
        y = []

        DOMINANT_EYE = get_participant_dominant_eye(participant)

        # for each packet, plot the gaze point
        for packet in GAZE_DATA_BOOK["data"]:
            if packet[f"{DOMINANT_EYE}_gaze_point_validity"] == 0:
                continue
            x.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][0] * X_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            y.append(
                (packet[f"{DOMINANT_EYE}_gaze_point_on_display_area"][1] * Y_PIXELS)
                # * DEGREES_PER_PIXEL
            )
            timestamps.append(packet[TIMESTAMP_IDENT])

        df = pd.DataFrame({"x": x, "y": y, "ts": timestamps})
        df = df.sort_values(by="ts")
        df = df.reset_index(drop=True)

        df["x"] = df["x"] * DEGREES_PER_PIXEL
        df["y"] = df["y"] * DEGREES_PER_PIXEL
        df["ts"] = df["ts"] / 1_000_000

        # display(df)
        SACCADIC_THRESHOLD = 80
        # Plot fixations
        fixations, v, labels = detect_fix_ivt(df, sacvel=SACCADIC_THRESHOLD)
        saccades_part = find_sacc_from_fix(fixations)
        saccades.append(saccades_part)

        print(
            f"Mean saccade length for participant {participant} in book {book.new_value}: {np.mean(saccades_part['len'])}"
        )

    # Do a t-test to see if the mean saccade length is significantly different between the two books
    # # Perform t-test
    # t_statistic, p_value = ttest_ind(saccades[0]['len'], saccades[0]['len'])

    # # Print the results
    # print(f"T-Statistic: {t_statistic}")
    # print(f"P-Value: {p_value}")

/Users/larsrefsgaard/Documents/Git/flow/apps/backend/velocityThreshold.py:62: RuntimeWarning: invalid value encountered in scalar divide
  rho = cov[0,1] / (sx*sy)


Mean saccade length for participant 9 in book Under the Tuscan Sun - : 0.020947408504164825
Mean saccade length for participant 9 in book Smart Farming Tech - : 0.023616154065117183
Mean saccade length for participant 10 in book Under the Tuscan Sun - : 0.02658998698764834
Mean saccade length for participant 10 in book Smart Farming Tech - : 0.022206601107762635
Mean saccade length for participant 11 in book Under the Tuscan Sun - : 0.02449691928250349
Mean saccade length for participant 11 in book Smart Farming Tech - : 0.022832409313730963
Mean saccade length for participant 12 in book Under the Tuscan Sun - : 0.027082794621441066
Mean saccade length for participant 12 in book Smart Farming Tech - : 0.0299278954134767
Mean saccade length for participant 13 in book Under the Tuscan Sun - : 0.02278718367582472
Mean saccade length for participant 13 in book Smart Farming Tech - : 0.019805045926132827
Mean saccade length for participant 14 in book Under the Tuscan Sun - : 0.0247298968531